In [1]:
from __future__ import division
import numpy as np
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
import math
import lmdb
import itertools
import json

# Make sure that caffe is on the python path:
caffe_root = '../../'  # this file is expected to be in {caffe_root}/examples/language_model
import sys
sys.path.insert(0, caffe_root + 'python')
import apollocaffe
sys.path.insert(0, caffe_root + 'python/apollocaffe/proto')
import caffe_pb2
sys.path.insert(0, caffe_root + 'examples/english_input_predictor2')
from train import (load_indice_file, forward)

In [2]:
config = json.load(open("config.json", 'r'))
net_config = config["net"]
apollocaffe.set_random_seed(config["solver"]["random_seed"])
apollocaffe.set_device(0)

2015-12-23 14:03:52 - GPU device 0


In [3]:
# Warning: load_idl returns an infinite generator. Calling list() before islice() will hang.
test_gen = load_indice_file(config["data"]["test_file"], config["net"])

In [23]:
net = apollocaffe.ApolloNet()
net.phase = 'test'
forward(net, test_gen.next(), config["net"], False)
net.load("/home/pig/apollocaffe-master/examples/english_input_predictor2/tmp/reinspect_200000.h5")

In [5]:
vocab = pickle.load(open(caffe_root + 'examples/english_input_predictor/vocab.pkl', 'rb'))
inv = lambda d: {v:k for k,v in d.iteritems()}
vocab_inv = inv(vocab)

In [12]:
def indice_2_string(indice):
    string = ''
    for i in indice:
        i = int(i)
        if i == net_config['unknown_symbol']:
            string += '#'
        elif i == net_config['start_symbol']:
            string += '$'
        elif i == net_config['zero_symbol']:
            string += ' '
        else:
            string += vocab_inv[i]
    return string

In [7]:
def get_max_indice(probs):
    indice = []
    for i in range(net_config['max_len']):
        peakIndex = np.argmax(probs[i,:])
        indice.append(peakIndex)
    return indice

In [17]:
def if_the_same(pred_indice, target_indice):
    for i in range(len(target_indice)):
        if target_indice[i]!=net_config['zero_symbol'] and target_indice[i]!=pred_indice[i]:
            return False
    return True

In [34]:
test_en = test_gen.next()
forward(net, test_en, config["net"], False)
print net.loss
forward(net, test_en, config["net"], True)

0.458062648773


In [35]:
accuracy = 0
for batch_id in range(net_config['batch_size']):
    data = test_en["input_words"]
    input_indice = data[batch_id,:,0,0]
    # print indice_2_string(input_indice)

    data = test_en["target_words"]
    target_indice = data[batch_id,:,0,0]
    # print indice_2_string(target_indice)

    probs = net.blobs['word_probs'].data
    out_prob = np.zeros((net_config['max_len'], net_config['vocab_size']))
    for i in range(net_config['max_len']):
        out_prob[i,:] = probs[batch_id+i*net_config['batch_size'],:]
    output_indice = get_max_indice(out_prob)
    # print indice_2_string(output_indice)
    if if_the_same(output_indice, target_indice):
        accuracy += 1
    else:
        print indice_2_string(input_indice)
        print indice_2_string(target_indice)
        print indice_2_string(output_indice)
print accuracy/net_config['batch_size']

$bill s and they didn't cha                                                                         
                           charge                                                                   
gon  iisaa ahatatid   ddui change                                                                   
$pokemom                                                                                            
        pokemom                                                                                     
gal     pokemon                                                                                     
$about me s don't say abit                                                                          
                          shit                                                                      
g aaaaaommosod  ddadssaaaaabout                                                                     
$s the reason j was rokd                                                                   

In [ ]:
forward(net, test_en, config["net"], False)
print net.loss
forward(net, test_en, config["net"], True)
for batch_id in range(10):
    data = test_en["input_words"]
    input_indice = data[batch_id,:,0,0]
    print indice_2_string(input_indice)

    data = test_en["target_words"]
    input_indice = data[batch_id,:,0,0]
    print indice_2_string(input_indice)

    probs = net.blobs['word_probs'].data
    out_prob = np.zeros((net_config['max_len'], net_config['vocab_size']))
    for i in range(net_config['max_len']):
        out_prob[i,:] = probs[batch_id+i*net_config['batch_size'],:]
    output_indice = get_max_indice(out_prob)
    print indice_2_string(output_indice)